#  Data Fusion All

En este notebook se procederá a la fusión del dataset limpiado en el notebook 1_clean_dataset (*hits*) con el generado en el 3 (canciones aleatorias), que es el que usaremos para entrenar nuestros modelos, para conformar un dataset de entrenamiento de para los modelos.

Para ello se concatenaran ambos dataframe y se eliminaran todas las duplicidades.
1. En caso de que el dataframe de *hits* tenga duplicidades, se ordenará por semanas en lista, y se eliminaran las duplicidades que menos tiempo tienen.
2. En caso de que dataframe de canciones aleatorias tenga duplicidades, nos quedaremos con la primera encontrada.
3. En caso de que haya una duplicidad de un hit con una canción aleatoria, se descartará la aleatoria.
4. Para la concatenación, y generar un dataset balanceado, se hará una selección del mismo tamaño que los *hits*, para así obtener un 50/50 de *hits* y no *hits*.
5. Añadiremos el *target* al dataset, que será 1 cuando es un *hit* y 0 cuando no.
6. Por último mezclamos todo el dataset para generar mayor aleatoriedad, y no tener los *hits* por un lado y las demás por otro.

Después, y tras los estudios realizados en el notebook 1_hit_analysis dentro del directorio Hit_Data_Analysis. Se puede apreciar un importante cambio en cuanto a géneros musicales en los *hits* se refiere, a partir del año 1993, se puede apreciar como la música urbana empieza a coger fuerza y ciertas características también varían. Por ello se ha decidido entrenar los modelos con un dataset que contenga *hits* y no hits desde esa fecha, ya que a partir de esa fecha es cuando se aprecian que los hits están más acordes a los de la actualidad, y por tanto cabe pensar que entrenando sobre este dataset podremos obtener una mayor capacidad de predicción de *hits* en la actualidad.

In [2]:
import pandas as pd
import numpy as np

In [2]:
df_random = pd.read_csv("../Data/random_songs_uniq_1958_2020.csv")

In [3]:
df_random.shape

(69067, 23)

In [4]:
df_random.head()

,artist,title,id,release_date,collaboration,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,time_signature,duration_ms,popularity_artist,popularity_song,genres,album,label
0,Airbourne,It Ain't Over Till It's Over,68SzcK5s3H4RXEyyJtdFAO,2010,False,0.484,0.999,9,-2.336,1,...,0.1590,0.0323,111.019,4,197706,59,33,"['glam metal', 'hard rock', 'metal', 'rock', '...",No Guts. No Glory,Roadrunner Records
1,Slow Meadow,Hurricane,55V1T1cUIWJYJPI77asAat,2017-11-17,False,0.117,0.264,2,-13.967,1,...,0.0762,0.0696,79.146,4,331008,54,0,"['calming instrumental', 'chamber pop', 'compo...",Costero,Hammock Music
2,Green Day,¿Viva La Gloria? (Little Girl),2mnk1GHN0XmznKiucY4ZW4,2009-05-15,False,0.519,0.845,5,-4.415,0,...,0.0922,0.5320,94.955,4,227680,82,55,"['permanent wave', 'pop punk', 'punk']",21st Century Breakdown,Reprise
3,Bart Wolffe,Three Questions by Leo Tolstoy,2gjKSPZulrHzqHQXmKksq9,2014-02-24,False,0.666,0.254,1,-15.736,1,...,0.1160,0.5090,75.985,4,654342,3,4,[],10 Minute Short Stories,Saland Publishing
4,Satin Jackets,Little Airplanes,5lzq0LoVd0y5YPLlc64FMz,2020-05-29,False,0.546,0.655,0,-7.540,1,...,0.1090,0.2090,117.001,4,330614,61,48,"['alternative dance', 'deep house', 'indie sou...",Little Airplanes,Eskimo Recordings


In [11]:
df_hits = pd.read_csv ("../Data/top_100_Billboard_1958_2020_to_train.csv",dtype={'song_lyrics':'string'})

Se ordena por mayor número de semanas en *hot-100* y se eliminan duplicados quedándonos con el primero

In [12]:
df_hits_uniq = df_hits.sort_values("weeks", ascending=False).drop_duplicates(subset = ["id"])
# las busquedas en Spotipy, a traves de artista y titulo, pueden encontrar la misma cancion con Ids diferentes,
# ya que la canción puede encontrarse en un album y como single y por tanto los ids no coincidiran aunque es
# la misma canción por ello nos aseguramos eliminando los duplicados que tengan mismo artista y titulo de la
# canción, para solventar esta casuistica.
df_hits_uniq = df_hits_uniq.drop_duplicates(subset = ["artist","title"], keep="first")

In [13]:
len(df_hits_uniq)

25507

Se realiza la concatenación, balanceado del dataset y se añade el *target*

In [14]:
df_concact = pd.concat([df_hits_uniq, df_random])
df_concact = df_concact.drop_duplicates(subset = "id", keep="first")
# las busquedas en Spotipy, a traves de artista y titulo, pueden encontrar la misma cancion con Ids diferentes,
# ya que la canción puede encontrarse en un album y como single y por tanto los ids no coincidiran aunque es
# la misma canción por ello nos aseguramos eliminando los duplicados que tengan mismo artista y titulo de la
# canción, para solventar esta casuistica.
df_concact = df_concact.drop_duplicates(subset = ["artist","title"], keep="first")

In [17]:
df_filter = df_concact.iloc[:2 * len(df_hits_uniq)]

In [18]:
df_target = df_filter.assign(target = np.where(pd.isnull(df_filter['rank']), 0, 1))

In [19]:
df_target.shape

(51014, 32)

Se mezcla todo el dataset, para dar aleatoriedad.

In [21]:
df_shuffle = df_target.sample(frac=1, random_state=34).reset_index(drop=True)

In [ ]:
pd.options.display.max_columns = None

In [22]:
df_shuffle.head()

,artist,title,id,year_chart,date_chart,release_date,collaboration,rank,weeks,isNew,peakPos,lastPos,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms,popularity_artist,popularity_song,genres,album,label,song_lyrics,target
0,Nat King Cole,Time And The River,7vu51HpremcwWKhBhVccDB,1960.0,1960-03-21,1965-08-01,False,99.0,8.0,False,30.0,85.0,0.415,0.249,10,-13.619,1,0.0295,0.893,0.000008,0.1190,0.3520,83.912,4,139573,68,13,"['adult standards', 'lounge', 'soul', 'vocal j...",Looking Back,CAPITOL CATALOG MKT (C92),<NA>,1
1,Keyshia Cole,Last Night,3Cn1FSSTXeQ7hCfrbizrD0,2007.0,2007-06-30,2007-01-01,True,39.0,22.0,False,10.0,31.0,0.916,0.849,3,-5.034,0,0.0639,0.166,0.000193,0.0859,0.9740,121.002,4,255773,71,57,"['dance pop', 'deep pop r&b', 'hip pop', 'neo ...",Just Like You,Geffen,"Last night, I couldn't even get an answer\nI t...",1
2,Jónsi,Around Us,1VYXp8Y1GUyY04wWhlAfeE,NaN,NaN,2010-11-29,False,NaN,NaN,NaN,NaN,NaN,0.324,0.641,3,-9.136,1,0.0411,0.105,0.009490,0.2290,0.0666,139.938,4,315120,55,13,"['compositional ambient', 'icelandic pop', 'po...",Around Us,Parlophone UK,NaN,0
3,Cassowary,She Funked Me,5x6KuClP6lLCvgjXUbLKGb,NaN,NaN,2019-11-19,False,NaN,NaN,NaN,NaN,NaN,0.741,0.646,2,-6.897,0,0.0314,0.234,0.793000,0.0926,0.6160,118.018,4,221426,31,5,[],She Funked Me,Fat Possum,NaN,0
4,Ludwig van Beethoven,"Beethoven: Symphony No. 2 in D Major, Op. 36: ...",6WKXJwgtSdkDj3fRga7VWy,NaN,NaN,2003,True,NaN,NaN,NaN,NaN,NaN,0.326,0.187,2,-19.562,1,0.0459,0.918,0.651000,0.2660,0.3090,78.314,4,373280,74,34,"['classical', 'classical era', 'early romantic...",Beethoven : Symphonies 1-9,Warner Classics,NaN,0


In [23]:
df_shuffle.to_csv("../Data/fusion_songs_1958_2020_to_train.csv", index = False, header=True)

# Dataset desde 1993-2020

A continuación se generará el dataset fusión, con canciones posteriores a 1993, el proceso es muy similar al anterior, con la diferencia de filtrar los datos previamente, para las canciones aleatorias, para los hits ya los hemos seleccionado en el cuaderno 1_clean_dataset.

In [5]:
df_hits_93_20 = pd.read_csv("../Data/top_100_Billboard_1993_2020_to_train.csv")

In [6]:
df_random_93_20 = df_random[df_random["release_date"]>="1993"]

In [7]:
df_random_93_20 = df_random_93_20.sample(frac=1, random_state=33).reset_index(drop=True)

In [8]:
df_concact_93_20 = pd.concat([df_hits_93_20, df_random_93_20])
df_concact_93_20 = df_concact_93_20.drop_duplicates(subset = "id", keep="first")
# las busquedas en Spotipy, a traves de artista y titulo, pueden encontrar la misma cancion con Ids diferentes,
# ya que la canción puede encontrarse en un album y como single y por tanto los ids no coincidiran aunque es
# la misma canción, por ello nos aseguramos eliminando los duplicados que tengan mismo artista y titulo de la
# canción, para solventar esta casuistica.
df_concact_93_20 = df_concact_93_20.drop_duplicates(subset = ["artist","title"], keep="first")

In [9]:
df_filter_93_20 = df_concact_93_20.iloc[:2 * len(df_hits_93_20)]

In [10]:
df_target_93_20 = df_filter_93_20.assign(target = np.where(pd.isnull(df_filter_93_20['rank']), 0, 1))

In [11]:
df_target_93_20.shape

(20462, 32)

In [12]:
df_shuffle_93_20 = df_target_93_20.sample(frac=1, random_state=34).reset_index(drop=True)

In [13]:
df_shuffle_93_20.to_csv("../Data/fusion_songs_1993_2020_to_train.csv", index = False, header=True)